In [29]:
import pandas as pd 
import numpy as np 
import neo4j


##import seaborn as sns
import warnings
from neo4j import GraphDatabase
warnings.filterwarnings('ignore')
import networkx as nx
from networkx.algorithms.link_analysis.pagerank_alg import pagerank
from collections import Counter
##from pandas_profiling import ProfileReport

In [30]:
url = "neo4j+s://9d0eb28d.databases.neo4j.io"
username = "neo4j"
password = "ZsZO3ya5YB8EcmVwmxYEqn6Amj_Z7R4t_6VDU5oMIeI"

driver = GraphDatabase.driver(url, auth=(username, password))
with driver.session() as session:
    result = session.run("MATCH (n) RETURN count(n)")
    count = result.single()[0]
    print(f"La conexión con Aura Neo4j se ha establecido correctamente. Hay {count} nodos en la base de datos.")

La conexión con Aura Neo4j se ha establecido correctamente. Hay 1163 nodos en la base de datos.


In [33]:
query = """
   MATCH (u:User {p : $id})-[:IN_VISTAS]->(m:Movie)-[:IN_GENRE]->(n:Genre)<-[:IN_GENRE]-(m2:Movie)
where m.id <> m2.id
return u.id, m.nombre, m2.nombre, n.nombre
"""
def _peliculas_cliente(tx, password):
    
    result = tx.run(query, id=password)
    return list(result)
def find_movies_of_year(password):
    records = []
    with driver.session() as session:
        result = session.read_transaction(_peliculas_cliente, password)
        for record in result:
            records.append(record)
    print("Found  {record} records ".format(record=len(records )))

    return records
a = find_movies_of_year(9487)

Found  1154 records 


In [35]:
def create_networkx_graph(password):
    
    val_count  = Counter([(res['m.nombre'],res['m2.nombre']) for res in find_movies_of_year(password) ])
    if len(val_count)==0:
        print(f" no records for {password}")
    print(f"  no of  records for {password} {len(val_count)}")

    recs = pd.DataFrame([[a[0], a[1], val_count[a]] for a in val_count])

    G = nx.DiGraph()
    #recs= recs.head(20)
    total = len(val_count)
    for idx , rec in recs.iterrows():
        #print(f"{idx} of {total} loaded")
        G.add_edge(rec[0], rec[1], weight = rec[2])
    return G
G= create_networkx_graph(password =9487)

Found  1154 records 
  no of  records for 9487 538


In [36]:
def run_page_rank(G, password):
    pr=pagerank(G)
    sorted_pr = {k: v for k, v in sorted(pagerank(G).items(), key=lambda item: item[1], reverse = True)}
    df= pd.DataFrame(sorted_pr.items()).assign(password= password)
    df['rank'] = df[1].rank(method='dense', ascending=False)
    print(f"done calculating page rank")
    return df
df = run_page_rank(G,1)

done calculating page rank


In [38]:
print(df.head(15))

                            0         1  password  rank
0             alles ist liebe  0.002074         1   1.0
1   shabab yarqos fawq al-nar  0.002074         1   1.0
2                 homewrecker  0.002074         1   1.0
3                     overdue  0.002074         1   1.0
4                   ai dao di  0.002074         1   1.0
5                      ho yuk  0.002074         1   1.0
6             husband hunters  0.002074         1   1.0
7               la governante  0.002074         1   1.0
8       fovou tous ellines...  0.002074         1   1.0
9        the princess diaries  0.002070         1   2.0
10            at the drive-in  0.002070         1   2.0
11   shakhe goli baraye aroos  0.002070         1   2.0
12      she's out of his mind  0.002070         1   2.0
13                hesapta ask  0.002070         1   2.0
14                     tallos  0.002070         1   2.0
